In [ ]:
import pandas as pd


# Análise de Vendas de derivados de petróleo

In [ ]:
vendasValor = pd.read_csv('/content/vendas-derivados-petroleo-etanol-m3-1990-2022.csv', sep=';', quotechar='"')
vendasValor = vendasValor.query('PRODUTO in ["GASOLINA DE AVIAÇÃO","QUEROSENE DE AVIAÇÃO"]')
vendasValor = vendasValor.query('ANO >= 2012')
vendasValor['VENDAS'] = pd.to_numeric(vendasValor['VENDAS'].str.replace(',','.'), errors='coerce')
vendasValor_agg = vendasValor.groupby('ANO')['VENDAS'].sum()
vendasValor_agg = round(vendasValor_agg.div(1000000),3)
vendasValor_agg


ANO
2012    7.368
2013    7.302
2014    7.546
2015    7.419
2016    6.822
2017    6.746
2018    7.213
2019    7.024
2020    3.585
2021    4.433
2022    2.853
Name: VENDAS, dtype: float64

# Análise de ocorrências

In [ ]:
from sqlalchemy import create_engine

# Create a dataframe
ocorrencia = pd.read_csv('/content/ocorrencia.csv', sep=';', quotechar='"')
ocorrencia_tipo = pd.read_csv('/content/ocorrencia_tipo.csv', sep=';', quotechar='"')

# Create a SQLite database and connect to it
engine = create_engine('sqlite:///data.db')

# Write the dataframe to a table in the database
ocorrencia.to_sql('ocorrencia', engine, if_exists='replace')
ocorrencia_tipo.to_sql('ocorrencia_tipo', engine, if_exists='replace')

In [ ]:
# Execute a the query to extract all ocorrencias
query = """
SELECT COUNT(*) AS 'Quantidade de ocorrências', strftime('%Y', date(substr(ocorrencia.ocorrencia_dia,7,4) || '-' || substr(ocorrencia.ocorrencia_dia,4,2) || '-' || substr(ocorrencia.ocorrencia_dia,1,2))) AS ANO
FROM ocorrencia
JOIN ocorrencia_tipo
ON ocorrencia.codigo_ocorrencia1 = ocorrencia_tipo.codigo_ocorrencia1
WHERE ocorrencia_tipo.ocorrencia_tipo = 'FALHA DO MOTOR EM VOO' 
GROUP BY strftime('%Y', date(substr(ocorrencia.ocorrencia_dia,7,4) || '-' || substr(ocorrencia.ocorrencia_dia,4,2) || '-' || substr(ocorrencia.ocorrencia_dia,1,2)));
"""
result = engine.execute(query)

# Fetch all rows from the result
allOcurrencies = result.fetchall()

# Print the rows
for row in allOcurrencies:
    print(row)

(72, '2012')
(72, '2013')
(74, '2014')
(65, '2015')
(59, '2016')
(45, '2017')
(53, '2018')
(58, '2019')
(87, '2020')
(82, '2021')
(38, '2022')


In [ ]:
# Execute a the query to extract only Acidentes and Incidentes graves
query = """
SELECT COUNT(*) AS 'Quantidade de ocorrências', strftime('%Y', date(substr(ocorrencia.ocorrencia_dia,7,4) || '-' || substr(ocorrencia.ocorrencia_dia,4,2) || '-' || substr(ocorrencia.ocorrencia_dia,1,2))) AS ANO
FROM ocorrencia
JOIN ocorrencia_tipo
ON ocorrencia.codigo_ocorrencia1 = ocorrencia_tipo.codigo_ocorrencia1
WHERE ocorrencia_tipo.ocorrencia_tipo = 'FALHA DO MOTOR EM VOO' AND (ocorrencia.ocorrencia_classificacao = 'ACIDENTE' OR ocorrencia.ocorrencia_classificacao = 'INCIDENTE GRAVE')
GROUP BY strftime('%Y', date(substr(ocorrencia.ocorrencia_dia,7,4) || '-' || substr(ocorrencia.ocorrencia_dia,4,2) || '-' || substr(ocorrencia.ocorrencia_dia,1,2)));
"""
result = engine.execute(query)

# Fetch all rows from the result
AcidentesIncidentesGraves = result.fetchall()

# Print the rows
for row in AcidentesIncidentesGraves:
    print(row)

(45, '2012')
(50, '2013')
(44, '2014')
(42, '2015')
(43, '2016')
(31, '2017')
(43, '2018')
(35, '2019')
(55, '2020')
(47, '2021')
(27, '2022')


# Análise de valores

In [ ]:
#Creation of the dataframes
df1 = pd.DataFrame(vendasValor_agg)
df2 = pd.DataFrame(allOcurrencies)
df3 = pd.DataFrame(AcidentesIncidentesGraves)



In [ ]:
df1.index

Int64Index([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022], dtype='int64', name='ANO')

In [ ]:
#Adjustments on the column names
df2 = df2.rename(columns={1:'ANO',0:'QTD OCORRENCIAS TOTAL'})
df2 = df2.set_index('ANO')
df2.index = df2.index.astype(int)
df3 = df3.rename(columns={1:'ANO',0:'QTD ACIDENTES E INCIDENTES GRAVES'})
df3 = df3.set_index('ANO')
df3.index = df3.index.astype(int)

In [ ]:
merged_df = df1.join(df2)
merged_df = merged_df.join(df3)
merged_df

,VENDAS,QTD OCORRENCIAS TOTAL,QTD ACIDENTES E INCIDENTES GRAVES
ANO,,,
2012,7.368,72,45
2013,7.302,72,50
2014,7.546,74,44
2015,7.419,65,42
2016,6.822,59,43
2017,6.746,45,31
2018,7.213,53,43
2019,7.024,58,35
2020,3.585,87,55


In [ ]:
#Calcula o valor da meta para o motivo FALHA DO MOTOR EM VOO
meta_filter =  merged_df.index <= 2016
meta_df = merged_df[meta_filter]
value_meta = meta_df['QTD ACIDENTES E INCIDENTES GRAVES'] / meta_df['VENDAS']
meta = round(value_meta.mean(),3)
meta

6.15

In [ ]:
#Calcula o valor real para o motivo FALHA DO MOTOR EM VOO
real_values = (merged_df.index <= 2021) & (merged_df.index > 2016)
real_values_df = merged_df[real_values]
value_2021_df = real_values_df['QTD ACIDENTES E INCIDENTES GRAVES'] / real_values_df['VENDAS']
value_2021 = round(value_2021_df.mean(),3)
value_2021

8.297